In [34]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

import geopandas as gpd
import shapely
from shapely.geometry import Point

import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

from descartes import PolygonPatch
import dill
import folium

# reference shapefile : elementary, secondary and unified school districts from census

In [20]:
#shapefile : census school district (note: NYC is considered one big bloc)

unsd_read = gpd.read_file('./Shapefiles/Census_schoolDistricts/tl_2015_36_unsd.shp').to_crs(epsg='4326')
#elsd_read = gpd.read_file('./Shapefiles/tl_2015_36_elsd.shp')
scsd_read = gpd.read_file('./Shapefiles/Census_schoolDistricts/tl_2015_36_scsd.shp').to_crs(epsg='4326')


In [ ]:
scsd_read[0:3]
unsd_read[0:3]

In [21]:
# 
sc_toAdd = scsd_read[scsd_read['NAME'].str.contains('Bellmore-Merrick|Sewanhaka|Valley Stream')][['NAME','GEOID','geometry']]
df_refSHP = unsd_read[['NAME','GEOID','geometry']].append(sc_toAdd)
#df_refSHP = unsd_read[['NAME','GEOID','geometry']]

# delete some data
#nysch=nysch[~nysch["NAME"].str.contains('School District Not Defined')]
#nysch=nysch[~nysch["NAME"].str.contains('Binghamton Common School District')]

df_refSHP['refshpindex'] = range(len(df_refSHP))
df_refSHP.set_index(['refshpindex'],inplace=True)
df_refSHP.rename(columns={'NAME':'name','GEOID':'geoid'},inplace=True)

In [22]:
refshape_json = []
for i in range(len(unsd_read)):
    refshape_json.append(unsd_read[i:i+1].to_json())
for i in range(len(scsd_read)):
    refshape_json.append(scsd_read[i:i+1].to_json())
df_refSHP['json']=refshape_json

In [23]:
# compute centroid
def getCentroid(shape):
    c=shape.centroid
    return (c.x,c.y)
df_refSHP['centroid'] = df_refSHP['geometry'].apply(lambda s: getCentroid(s))
t=zip(*df_refSHP['centroid'].tolist())
df_refSHP['centroid_long']=t[0]
df_refSHP['centroid_lat']=t[1]


In [13]:
df_refSHP[:3]


,name,geoid,county,centroid,centroid_long,centroid_lat,geometry,json
0,North Shore Central School District,3626370,Nassau,"(-73.6239251152, 40.8362893218)",-73.623925,40.836289,"POLYGON ((-73.67107299999999 40.859299, -73.65...","{""type"": ""FeatureCollection"", ""features"": [{""g..."
1,Seaford Union Free School District,3626400,Nassau,"(-73.4887751411, 40.5975612621)",-73.488775,40.597561,"POLYGON ((-73.50683099999999 40.666806, -73.50...","{""type"": ""FeatureCollection"", ""features"": [{""g..."
2,Uniondale Union Free School District,3629280,Nassau,"(-73.5972693079, 40.7139621285)",-73.597269,40.713962,"POLYGON ((-73.619737 40.68251799999999, -73.61...","{""type"": ""FeatureCollection"", ""features"": [{""g..."


In [24]:
# add county info
countyshape_read= gpd.read_file('./Shapefiles/NYS_GIS_civil/Counties.shp').to_crs(epsg='4326')
df_countyshape = countyshape_read[['NAME','geometry']]

In [ ]:
countyshape_read[:3]


In [25]:
# assign each school district to a county region

def findEncompassingShape(point,shapelist):
    for refshape_ind,refshape in shapelist:
        if refshape.contains(point):
            return refshape_ind
    return -1

ref_i_centroids = zip(df_refSHP.index,df_refSHP['centroid'])
county_shapes = zip(df_countyshape.index,df_countyshape['geometry'])
index_map = []

for refindex,centroid in ref_i_centroids:
    p = Point(centroid[0],centroid[1])
    county_index = findEncompassingShape(p,county_shapes)
    index_map.append((refindex,county_index))

#index_map

In [26]:
# join county to school data
df_assign=pd.DataFrame(index_map,columns=['oldind','targetind'])
t1=pd.merge(df_refSHP,df_assign,left_index=True,right_on='oldind',how='left')
t2=pd.merge(t1,df_countyshape,left_on='targetind',right_index = True,how='left')

#df_crimerate_refshape = t2[['name','County','PD','crimerate_total','crimerate_violent','crimerate_nonviolent']]
print len(t1)
t2.rename(columns={'geometry_x':'geometry','NAME':'county'},inplace=True)
df_refSHP = t2[['name','geoid','county','centroid','centroid_long','centroid_lat','geometry','json']]
df_refSHP[:3]


672


,name,geoid,county,centroid,centroid_long,centroid_lat,geometry,json
0,North Shore Central School District,3626370,Nassau,"(-73.6239251152, 40.8362893218)",-73.623925,40.836289,"POLYGON ((-73.67107299999999 40.859299, -73.65...","{""type"": ""FeatureCollection"", ""features"": [{""g..."
1,Seaford Union Free School District,3626400,Nassau,"(-73.4887751411, 40.5975612621)",-73.488775,40.597561,"POLYGON ((-73.50683099999999 40.666806, -73.50...","{""type"": ""FeatureCollection"", ""features"": [{""g..."
2,Uniondale Union Free School District,3629280,Nassau,"(-73.5972693079, 40.7139621285)",-73.597269,40.713962,"POLYGON ((-73.619737 40.68251799999999, -73.61...","{""type"": ""FeatureCollection"", ""features"": [{""g..."


In [27]:
# send to local database
df_refSHP[['name','centroid','centroid_lat','centroid_long','json']].to_sql('refshape', local_engine_alchemy, if_exists='replace',index_label='refshpindex')

In [ ]:
dill.dump(df_refSHP, open('../OUTDATA/refshape_sch2015.dill','wb'))

In [35]:
df_refSHP=dill.load(open('../OUTDATA/refshape_sch2015.dill','rb'))

In [28]:
df_refSHP['name'][df_refSHP['name'].str.contains('Scarsdale|Eastchester|Tuckahoe|White Plains|Yonkers|Bronxville|New Rochelle|Greenburg|Edgemont')]


106      Scarsdale Union Free School District
172           Tuckahoe Common School District
199              Yonkers City School District
219     Bronxville Union Free School District
231       Edgemont Union Free School District
368         White Plains City School District
369        Greenburgh Central School District
449    Eastchester Union Free School District
453         New Rochelle City School District
656       Tuckahoe Union Free School District
Name: name, dtype: object

# Set the reference geometry that everything will be overlayed on.
#### currently using school geometry  (can be other shapes)

In [ ]:
# use school geometry as the reference (can be other shapes)
refSHP=df_refSHP['geometry'].tolist()

### school performance data (with shapefile index assigned by addSchDistrict.pl)

In [38]:
# for initial matching to NYS school performance data
df_refSHP['name'].to_csv('./NYSED_3-8gradeAssess_2014-2015/sdNames.census_shx.csv')

In [39]:
performance=pd.read_table('./NYSED_3-8gradeAssess_2014-2015/3-8_ELA_AND_MATH_RESEARCHER_FILE_2015.DISTRICTSwSHPassignment.tab')
performance['SID']=performance['SID'].astype(int)
performance['MEAN_SCALE_SCORE']=pd.to_numeric(performance['MEAN_SCALE_SCORE'],errors=coerce)
performance['L4_PCT']=pd.to_numeric(performance['L4_PCT'],errors=coerce)


In [ ]:
performance.columns

In [40]:
# choose here the school metric and aggregator
t=performance[['SID','SD_NAME','L4_PCT']][performance['ITEM_DESC'].str.contains('Grade 8 Math')]
t.rename(columns={'SID':'refshpindex','L4_PCT':'sch_perform'}, inplace=True)
df_schoolPerform=t.groupby('refshpindex').mean()
df_schoolPerform[:3]

,sch_perform
refshpindex,
0,0.0
1,10.0
2,0.0


In [41]:
df_schoolPerform_shape = pd.merge(df_schoolPerform,df_refSHP,left_index=True,right_index=True)
df_schoolPerform_shape[:3]

,sch_perform,name,geoid,county,centroid,centroid_long,centroid_lat,geometry,json
0,0.0,North Shore Central School District,3626370,Nassau,"(-73.6239251152, 40.8362893218)",-73.623925,40.836289,"POLYGON ((-73.67107299999999 40.859299, -73.65...","{""type"": ""FeatureCollection"", ""features"": [{""g..."
1,10.0,Seaford Union Free School District,3626400,Nassau,"(-73.4887751411, 40.5975612621)",-73.488775,40.597561,"POLYGON ((-73.50683099999999 40.666806, -73.50...","{""type"": ""FeatureCollection"", ""features"": [{""g..."
2,0.0,Uniondale Union Free School District,3629280,Nassau,"(-73.5972693079, 40.7139621285)",-73.597269,40.713962,"POLYGON ((-73.619737 40.68251799999999, -73.61...","{""type"": ""FeatureCollection"", ""features"": [{""g..."


In [ ]:
with open('test.json','wb') as f: 
    f.write(df_schoolPerform_shape['json'][0:1].to_string)

# plot school performance on map

In [46]:
crs = crs = {'init': 'epsg:4326', 'no_defs': True}
geo_df = gpd.GeoDataFrame(df_schoolPerform_shape, crs=crs)
geo_str = geo_df.to_json()

In [ ]:
print geo_df['sch_perform'].describe()

In [36]:
# function to provide color (from colorbrewer)
def getColor(d):
    if d > 4: 
        return '#313695' # blue
    elif d > 3:
        return '#4575b4'
    elif d > 2.5:
        return '#74add1'
    elif d > 2:
        return '#abd9e9'
    elif d > 1.5:
        return '#e0f3f8'
    elif d > 1:
        return '#fee090'
    elif d > .75:
        return '#fdae61'
    elif d > .5:
        return '#f46d43'
    elif d > .25:    
        return '#d73027'
    elif d >= 0 : 
        return '#a50026' # red
    return '#d9d9d9'


In [44]:
#
m = folium.Map([40.6,-73.938], zoom_start=8,tiles='cartodbpositron')

folium.GeoJson(geo_df,    style_function=lambda feature: {
        'fillColor': getColor(feature['properties']['sch_perform']),
        'color' : 'black',
        'weight' : 1,
        'fillOpacity':1, 'opacity':0.2
        }
              ).add_to(m)
path='schoolPerform.html'
m.save(path)


In [48]:
with open('../flask_actual/templates/refshape_school.json','wb') as f:
    f.write(geo_str)

In [ ]:
# matplotlib only works up to 999 points -> skip points if >999
def plotGeometry(shape):
    fig=plt.figure()
    ax = fig.add_subplot(111)
    x,y = shape.exterior.xy 
    coords = shape.exterior.coords
    N=len(x)
    factor = N//1000 +1
    shapepatch = PolygonPatch(shape)
    ax.plot(x[::factor],y[::factor]) 
    ax.add_patch(shapepatch)
#plotGeometry(schoolScore.at[8259,'geometry'])

In [ ]:
# matplotlib
# matplotlib only works up to 999 points -> skip points if >999
def plotGeometries(shapelist):
    fig=plt.figure()
    ax = fig.add_subplot(111)
    for shape in shapelist:
        x,y = shape.exterior.xy 
        N=len(x)
        factor = N//1000 +1
        #shapepatch = PolygonPatch(shape)
        ax.plot(x[::factor],y[::factor], color='black') 
        #ax.add_patch(shapepatch)

In [ ]:
# bokeh
from bokeh.plotting import show,figure,output_file
from bokeh.io import output_notebook
#output_notebook()


In [ ]:
for shape in refSHP:
    if not isinstance(shape,shapely.geometry.polygon.Polygon):
        print type(shape)
        

In [ ]:
def plotGeometries_bk(shapelist):
    output_file("lines.html", title="shapefile plot example")
    p = figure(title="New York State Locales (by school districts)", x_axis_label='longitude', y_axis_label='latitude')    
    for shape in shapelist:
        
        if isinstance(shape,shapely.geometry.polygon.Polygon):
            x,y = shape.exterior.xy 
            N=len(x)
            factor = N//2000 +1
            p.line(x[::factor],y[::factor],color='black')
    show(p)
plotGeometries_bk(refSHP)

### shapefile : zipcodes from census

In [ ]:
zc_read=gpd.read_file('./Shapefiles/tl_2015_us_zcta510.shp')


In [ ]:
zc_read['ZCTA5CE10']=zc_read['ZCTA5CE10'].astype(int)


In [ ]:
# new york zipcode ranges from https://data.ny.gov
nyzc=county[ ((county['ZCTA5CE10'] <=14925) & (county['ZCTA5CE10'] >= 10000)) | (county['ZCTA5CE10'] == 501) | (county['ZCTA5CE10'] == 544)| (county['ZCTA5CE10'] == 6390)][['GEOID10','ZCTA5CE10','geometry']]

In [ ]:
sz=nyzc[nyzc['ZCTA5CE10']==10583]

In [ ]:
ss.plot()


In [ ]:
ss_SHP=df_refSHP[df_refSHP['NAME'].str.contains('Scarsdale')]
#.iat[0,0]
#ss['geometry'].plot()
print ss_SHP['geometry']

In [ ]:
sz.plot()


In [ ]:
from geopandas.tools import overlay

# Pricing data; assigned to corresponding refshape

In [ ]:
df_zillow_ny_regionData['refSHPindex']=assign_zRegion2RefSHP
df_zillow_ny_regionData[:3]

In [ ]:
# sanity check: zilow regions not mappable to schools (because sec-schools are omitted)
df_zillow_ny_regionData[df_zillow_ny_regionData['refSHPindex']==-1]

In [ ]:
# combine data in the same refSHPindex using desired aggregator
df_price=df_zillow_ny_regionData[['refSHPindex','ZPRICE','ZLAT','ZLONG']].groupby('refSHPindex').mean()
df_price.rename(columns={'ZPRICE':'zprice'},inplace=True)
df_price[:5]

# plot home prices on map

In [ ]:
t1=pd.merge(df_refSHP,df_price,left_index=True,right_index=True,how='left')
df_price_shape = t1[['name','centroid','geometry','zprice']]

In [ ]:
crs = crs = {'init': 'epsg:4326', 'no_defs': True}
geo_df = gpd.GeoDataFrame(df_price_shape, crs=crs)
geo_str = geo_df.to_json()

In [ ]:
print geo_df['zprice'].describe()

In [45]:
# function to provide color (from colorbrewer)
def getColor(d):
    if d > 1000000: 
        return '#313695' # blue
    elif d > 500000:
        return '#4575b4'
    elif d > 400000:
        return '#74add1'
    elif d > 300000:
        return '#abd9e9'
    elif d > 200000:
        return '#e0f3f8'
    elif d > 100000:
        return '#fee090'
    elif d > 80000:
        return '#fdae61'
    elif d > 60000:
        return '#f46d43'
    elif d > 40000:    
        return '#d73027'
    elif d > 20000: 
        return '#a50026' # red
    return '#d9d9d9'


In [ ]:
#
m = folium.Map([40.6,-73.938], zoom_start=8,tiles='cartodbpositron')

folium.GeoJson(geo_df,    style_function=lambda feature: {
        'fillColor': getColor(feature['properties']['zprice']),
        'color' : 'black',
        'weight' : 1,
        'fillOpacity':1, 'opacity':0.2
        }
              ).add_to(m)
path='medHomePrice.html'
m.save(path)


# get baseline pricing based on lat-long (kneighbor)

In [ ]:
# training set: data from 532 price,(zlat,zlong data)
t=df_price[df_price.index.values>-1]
X_train = t[['ZLONG','ZLAT']].as_matrix()
y_train=t['zprice'].tolist()

In [ ]:
# applied to: 672 refshapes
X_test = [ list(x) for x in df_refSHP['centroid'].as_matrix() ]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
kr = KNeighborsRegressor(n_neighbors=15)
kr.fit(X_train,y_train)

In [ ]:
price_baseline = kr.predict(X_test)

In [ ]:
df_price_baseline = pd.DataFrame({'price_baseline':price_baseline},index =range(672))
df_price_baseline[:3]


In [ ]:
dill.dump(df_price_baseline,open('../OUTDATA/price_baseline.dill','wb'))

# Combining pricing data and feature(s) based on refshapes

In [ ]:
print "price",len(df_price),df_price.index[:5]
print "school",len(df_schoolPerform),df_schoolPerform.index[:5]
print "crime",len(df_crime),df_crime.index[:5]
print "roi",len(df_roi),df_roi.index[:5]

In [ ]:
df_crime = dill.load(open('../OUTDATA/refshape_crime2014.dill','rb'))
df_crime[df_crime.index.values==1]

In [ ]:
df_roi=dill.load(open('../OUTDATA/refshape_roi_2006-2016.dill','rb'))
df_roi[:2]

In [ ]:
t1=pd.merge(df_price,df_schoolPerform,left_index=True,right_index=True,how='inner')
del t1['ZLAT']
del t1['ZLONG']
t2=pd.merge(t1,df_crime[['crimerate_total','crimerate_violent','crimerate_property']],left_index=True,right_index=True,how='inner')
t3=pd.merge(t2,df_roi[['roi']],left_index=True,right_index=True,how='inner')
t4=pd.merge(t3,df_price_baseline,left_index=True,right_index=True,how='inner')
t4[:10]
df_price_features=t4
df_price_features['premium'] = df_price_features['zprice']-df_price_features['price_baseline']
df_price_features[:3]

In [ ]:
plt.plot(df_price_features['price_baseline'], df_price_features['zprice'], '.')
plt.locator_params(nbins=6)
plt.xlabel('Baseline Home Price (lat-long model)')
plt.ylabel('Actual Home Price')

# posting to postgresql

In [14]:
from sqlalchemy import create_engine
from sqlalchemy import types as sqt

In [15]:
# connecting to local using sqlachemy
local_engine_alchemy = create_engine('postgresql://enghuiy:@localhost:5432/nysRealEstate')

In [ ]:
# remote db (heroku-app: homevalue-hunter)

remote_engine = create_engine('postgresql://%s:%s@%s:5432/%s' %(USER,PGPASSWORD,HOST,DATABASE))
try:
    connection=remote_engine.connect()
except:
    print "I am unable to connect to the database"

#result = connection.execute("select * from price2features" )

In [ ]:
df_price_features.to_sql('price2features', local_engine_alchemy, if_exists='replace',index_label='refshpindex')


In [ ]:
# save to files

df_price_features.to_csv('../OUTDATA/price2features.csv')

# extracting from postgresql

In [ ]:
import psycopg2
import urlparse


In [ ]:
# connect to local database using psycopg2
try:
    connl = psycopg2.connect("dbname='nysRealEstate' user='enghuiy' host='localhost' password=''")
except:
    print "I am unable to connect to the database"
connl.set_session(autocommit=True);   
cur = connl.cursor()

In [ ]:
# to use on heroku
url = urlparse.urlparse(url_)

conn = psycopg2.connect(
    database=url.path[1:],
    user=url.username,
    password=url.password,
    host=url.hostname,
    port=url.port,
#    sslmode='require'
)

# Machine Learning 

In [ ]:
import numpy as np
import re
import math

from sklearn import linear_model as lm
from sklearn.metrics import r2_score

In [ ]:
# normalization
def norm(x_in,x_norm):
    
    x_mu = np.mean(x_in)
    x_range = np.amax(x_in) - np.amin(x_in)
    x_norm [:] = [ ( x - x_mu ) / float (x_range) for x in x_in]
    return (x_mu, x_range)

# convert back to abs value
def unnorm(x_mu, x_range, x_norm):
    x_out=[]
    x_out [:] = [ x*x_range+x_mu for x in x_norm ]
    return x_out

In [ ]:
# extract data from dataframes
tmp=df_price_features.dropna()
print len(tmp)
#feature1=tmp['SCH_PERFORM'].tolist()
#homevalue=tmp['ZPRICE'].tolist()
tmp[:3]

In [ ]:
# univariate regression: school-performance; no baseline adjustment
X_train = np.asarray(tmp['sch_perform'].tolist()).reshape(len(tmp),1)
y_train = tmp['zprice'].tolist()

lm1 = lm.LinearRegression(normalize=True)
lm1.fit(X_train, y_train)
y_predict0 = lm1.predict(X_train)
score = lm1.score(X_train,y_train)

plt.plot(X_train, y_train, '.')
plt.plot(X_train, y_predict0)
plt.locator_params(nbins=6)
plt.xlabel('School Performance (% in top quantile; Gr8 math)')
plt.ylabel('Home Price')
print "R^2 = %5.2f" % score

#test_xy = df_price_features[['sch_perform','zprice']].dropna()
#X_test = np.asarray(test_xy['sch_perform'].tolist()).reshape(len(tmp),1)
#y_test = test_xy['zprice'].tolist()
#print len(y_test)

#y_predict = lm1.predict(y_test)


In [ ]:
# univariate regression: school-performance; with baseline adjustment
X_train = np.asarray(tmp['sch_perform'].tolist()).reshape(len(tmp),1)
y_train = tmp['premium'].tolist()

lm1 = lm.LinearRegression(normalize=True)
lm1.fit(X_train, y_train)
y_predict0 = lm1.predict(X_train)
score = lm1.score(X_train,y_train)

plt.plot(X_train, y_train, '.')
plt.plot(X_train, y_predict0)
plt.locator_params(nbins=6)
plt.xlabel('School Performance (% in top quantile; Gr8 math)')
plt.ylabel('Home Price - Baseline')
print "R^2 = %5.2f" % score

#test_xy = df_price_features[['sch_perform','zprice']].dropna()
#X_test = np.asarray(test_xy['sch_perform'].tolist()).reshape(len(tmp),1)
#y_test = test_xy['zprice'].tolist()
#print len(y_test)

#y_predict = lm1.predict(y_test)


In [ ]:
# multivariate regression: school,roi,crime; no baseline adjustment
X_train = np.asarray(tmp[['sch_perform','crimerate_total','roi']].as_matrix()).reshape(len(tmp),3)
y_train = tmp['zprice'].tolist()

lm3 = lm.LinearRegression(normalize=True)
lm3.fit(X_train, y_train)
y_predict0 = lm3.predict(X_train)
score = lm3.score(X_train,y_train)

#plt.plot(X_train, y_train, '.')
#plt.plot(X_train, y_predict0)
#plt.locator_params(nbins=6)
#plt.xlabel('School Performance (% in top quantile; Gr8 math)')
#plt.ylabel('Home Price')
print "R^2 = %5.2f" % score
print lm3.coef_


#test_xy = df_price_features[['sch_perform','zprice']].dropna()
#X_test = np.asarray(test_xy['sch_perform'].tolist()).reshape(len(tmp),1)
#y_test = test_xy['zprice'].tolist()
#print len(y_test)

#y_predict = lm1.predict(y_test)


In [ ]:
# univariate regression: school,crime,roi; with baseline adjustment
X_train = np.asarray(tmp[['sch_perform','crimerate_total','roi']].as_matrix()).reshape(len(tmp),3)
y_train = tmp['premium'].tolist()

lm3b = lm.LinearRegression(normalize=True)
lm3b.fit(X_train, y_train)
y_predict0 = lm3b.predict(X_train)
score = lm3b.score(X_train,y_train)

#plt.plot(X_train, y_train, '.')
#plt.plot(X_train, y_predict0)
#plt.locator_params(nbins=6)
#plt.xlabel('School Performance (% in top quantile; Gr8 math)')
#plt.ylabel('Home Price - Baseline')
print "R^2 = %5.2f" % score
print lm3b.coef_

#test_xy = df_price_features[['sch_perform','zprice']].dropna()
#X_test = np.asarray(test_xy['sch_perform'].tolist()).reshape(len(tmp),1)
#y_test = test_xy['zprice'].tolist()
#print len(y_test)

#y_predict = lm1.predict(y_test)


In [ ]:
# univariate regression (homePrice vs. SAT score)
x1_norm=[]; y_norm=[]
(x1_mu,x1_range) = norm(feature1,x1_norm)
(y_mu, y_range) = norm(homevalue, y_norm)

# univariate linear regression, with x0=1
#X_train = zip( np.ones(len(x1_norm)),x1_norm)

X_train = np.asarray(zip( np.ones(len(x1_norm)),x1_norm))
#X_train = zip(np.asarray(x1_norm).reshape(-1,1)



In [ ]:
# Create linear regression object
regr = lm.LinearRegression()
regr.fit(X_train, y_norm)

# convert y back to abs value
y_predicted_norm1 = regr.predict(X_train)
y_predicted1 = unnorm(y_mu,y_range,y_predicted_norm1)

print r2_score(y_norm, y_predicted_norm1)


In [ ]:
cur = connl.cursor()
#command=SELECT 'ZPRICE' from price2features
#cur.execute("""SELECT version()""")
#cur.fetchone()
#cur.execute("""SELECT refSHP."NAME","ZPRICE","SCH_PERFORM" from price2features JOIN refshape ON  price2features."REFSHPINDEX"=refSHP."refSHPindex" WHERE "SCH_PERFORM" > 0;""")
#cur.execute("""SELECT "ZPRICE","SCH_PERFORM" from price2features;""")
#cur.execute("""SELECT "ZPRICE","SCORE" from price2features WHERE "SCORE" IS NOT NULL""")
#data=zip(*cur.fetchall())
#homevalue = list(data[0])
#features  = list(data[1])

In [ ]:
# normalization
def norm(x_in,x_norm):
    
    x_mu = np.mean(x_in)
    x_range = np.amax(x_in) - np.amin(x_in)
    x_norm [:] = [ ( x - x_mu ) / float (x_range) for x in x_in]
    return (x_mu, x_range)

# convert back to abs value
def unnorm(x_mu, x_range, x_norm):
    x_out=[]
    x_out [:] = [ x*x_range+x_mu for x in x_norm ]
    return x_out

# univariate regression
def linearRegression(features,homevalue):
    x_norm=[]; y_norm=[]
    (x_mu,x_range) = norm(features,x_norm)
    (y_mu, y_range) = norm(homevalue, y_norm)

    X_train = np.asarray(zip( np.ones(len(x_norm)),x_norm))

    # Create linear regression object
    regr = lm.LinearRegression()
    regr.fit(X_train, y_norm)
    # convert y back to abs value
    y_predicted_norm = regr.predict(X_train)
    coeffs = regr.coef_
    intercepts = regr.intercept_

    y_predicted = unnorm(y_mu,y_range,y_predicted_norm)

    r2=r2_score(y_norm, y_predicted_norm)
     
    return (coeffs,intercepts,r2,y_predicted)


In [ ]:
coeffs,intercepts,r2,ypredicted =  linearRegression(features,homevalue)
